# Information
Predict if a customer leave the bank:  
Churn rate of a bank: the annual percentage rate at which customers stop subscribing to a service.  

The columns in order:   
Row Number  
Customer ID  
Lastname  
Credit Score  
Geography  
Gender  
Age  
Tenure  
Balance  
Number of Products  
Has Credit Card  
Is Active Member  
Estimated Salary  
Exited: They wait 6 months and enter the exit information for each customer.  

You can apply this model to any dataset with lots of variables and two outcome : 0 or 1

This is a classification problem


# Part 1: Data Preprocessing


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# 1) IMPORT THE DATASET
dataset = pd.read_csv('Churn_Modelling.csv')

X = dataset.iloc[:, 3:13].values # we excluded the row number, customer id, surname columns. they have no impact for a customer to decide to leave or stay. 
y = dataset.iloc[:, 13].values #the targets, actual outputs
#.values: runs only the values, no headings.

# Classification template

#According to template, after importing the dataset, and creating our dependent and independent variables
#we need to split the dataset into the training and test sets, however for this dataset
#we, first, need to encode the categorical variables:

# 2) ENCODING CATEGORICAL DATA: CATEGORICAL TEMPLATE
#For this we have categorical template. This section will endcode any categorical data that you have in your dataset:

# Encoding the Independent Variable: we have two categorical variables
#country and gender

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
#Country column number is 1. This will encode the values as 0, 1, 2...
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
#Gender column number is 2.
#we are creating dummy variables for the country column
#the reason for that we don't want any numbers except 0 or 1. 
#so we will say for example France = 0,0,1; Spain= 1,0,0; German; 0,1,0 instead of  0,1,2
#one hot encoder will create three dummy columns, place it before the credit score column.
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
#we don't need three variables to represent the three different countries, so
#we will delete one dummy variable to avoid falling into the dummy variable trap.
#this won't change the country information: France becomes = 0,0; German=0,1; Spain,1,0
X = X[:, 1:] #this takes all the rows in X (inputs), and takes columns from 1 to end
#CHANGED: just the column numbers, and one delete one dummy variable

# 3) SPLITTING THE DATASET INTO TRAINING AND TEST:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
#CHANGED: just the test size from 0.25 to 0.2 since we have 10,000 observations
#to avoid warning: we changed from sklearn.cross_validation to model_selection

# 4) FEATURE SCALING: 
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
#CHANGED: No change

# EVALUATING THE ANN
# Part 2: create the ANN with k_fold cross validation
#### takes approximately 30 min

In [2]:
#evaluating the ANN:
#k-fold cross validation improves the model. 
#each time we run the model we have different train and test accuracy. the variance between each run is a problem
#k-fold solves this
#k-fold split the training set into 10 fold, and we train our model on 9 fold and we test it on the last remaining fold
#k-fold belongs to scikit learn


from keras.wrappers.scikit_learn import KerasClassifier #we can apply scikit in our keras with this
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
#this are all the same with what we did in part 2, except the fit part in part 2
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs =100)
#BUILD_FN: builds the model; we give our function to it :build_classifier

#we use k-fld to retrun relevant measure of accuracy so let's define it
accuracies = cross_val_score(estimator = classifier, X =X_train, y=y_train, cv=10, n_jobs =-1)
#estimator: object to use to fit the data
#cv: thenumber of folds/ the k, in the k_fold
#n_jobs = number of CPUs to use to do the computation. -1 means all CPUs


Using TensorFlow backend.


Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 3s 453us/step - loss: 0.4936 - acc: 0.7957
Epoch 2/100
7200/7200 [==============================] - 3s 460us/step - loss: 0.4916 - acc: 0.7976
Epoch 2/100
7200/7200 [==============================] - 4s 495us/step - loss: 0.4943 - acc: 0.7969
Epoch 2/100
7200/7200 [==============================] - 3s 405us/step - loss: 0.4276 - acc: 0.7967
Epoch 3/100
7200/7200 [==============================] - 3s 427us/step - loss: 0.4283 - acc: 0.7971
Epoch 3/100
7200/7200 [==============================] - 3s 423us/step - loss: 0.4219 - acc: 0.7975
Epoch 4/100
7200/7200 [==============================] - 3s 419us/step - loss: 0.4193 - acc: 0.8218
Epoch 4/100
7200/7200 [==============================] - 3s 429us/step - loss: 0.4216 - acc: 0.8054
Epoch 4/100
7200/7200 [==============================] - 3s 398us/step - loss: 0.4172 - acc: 0.8124
Epoch 5/100
7200/7200 [==============================] - 3s 422u

/Users/nevinyilmaz/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.127717). Check your callbacks.
  % delta_t_median)


7200/7200 [==============================] - 3s 394us/step - loss: 0.4113 - acc: 0.8289
Epoch 6/100
7200/7200 [==============================] - 3s 384us/step - loss: 0.4114 - acc: 0.8301
Epoch 7/100
7200/7200 [==============================] - 3s 399us/step - loss: 0.4081 - acc: 0.8312
Epoch 7/100
7200/7200 [==============================] - 3s 461us/step - loss: 0.4067 - acc: 0.8340
Epoch 8/100
7200/7200 [==============================] - 3s 469us/step - loss: 0.4061 - acc: 0.8331
Epoch 8/100
7200/7200 [==============================] - 2s 347us/step - loss: 0.4057 - acc: 0.8342
Epoch 9/100
7200/7200 [==============================] - 2s 331us/step - loss: 0.4045 - acc: 0.8332
Epoch 9/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.4040 - acc: 0.8339
Epoch 9/100
7200/7200 [==============================] - 2s 334us/step - loss: 0.4036 - acc: 0.8342
Epoch 10/100
7200/7200 [==============================] - 2s 310us/step - loss: 0.4029 - acc: 0.8353
Epoch 10/10

7200/7200 [==============================] - 3s 349us/step - loss: 0.3953 - acc: 0.8343
Epoch 23/100
7200/7200 [==============================] - 3s 412us/step - loss: 0.3935 - acc: 0.8375
Epoch 24/100
7200/7200 [==============================] - 3s 400us/step - loss: 0.3950 - acc: 0.8351
Epoch 24/100
7200/7200 [==============================] - 3s 409us/step - loss: 0.3954 - acc: 0.8368
Epoch 24/100
7200/7200 [==============================] - 4s 522us/step - loss: 0.3949 - acc: 0.8369
Epoch 25/100
7200/7200 [==============================] - 4s 541us/step - loss: 0.3944 - acc: 0.8354
Epoch 25/100
7200/7200 [==============================] - 4s 488us/step - loss: 0.3940 - acc: 0.8383
Epoch 26/100
7200/7200 [==============================] - 3s 451us/step - loss: 0.3950 - acc: 0.8381
Epoch 26/100
Epoch 26/100
7200/7200 [==============================] - 3s 460us/step - loss: 0.3956 - acc: 0.8381
Epoch 26/100
7200/7200 [==============================] - 3s 367us/step - loss: 0.3932 - ac

...

7200/7200 [==============================] - 2s 315us/step - loss: 0.3984 - acc: 0.8351
Epoch 60/100
7200/7200 [==============================] - 2s 303us/step - loss: 0.4042 - acc: 0.8329
Epoch 61/100
7200/7200 [==============================] - 2s 315us/step - loss: 0.4007 - acc: 0.8329
Epoch 61/100
7200/7200 [==============================] - 2s 308us/step - loss: 0.4017 - acc: 0.8356
Epoch 61/100
7200/7200 [==============================] - 2s 305us/step - loss: 0.3987 - acc: 0.8357
Epoch 61/100
7200/7200 [==============================] - 2s 312us/step - loss: 0.4040 - acc: 0.8311
Epoch 62/100
7200/7200 [==============================] - 2s 298us/step - loss: 0.4015 - acc: 0.8349
Epoch 62/100
7200/7200 [==============================] - 2s 299us/step - loss: 0.4043 - acc: 0.8337
Epoch 63/100
7200/7200 [==============================] - 2s 300us/step - loss: 0.4006 - acc: 0.8343
Epoch 63/100
7200/7200 [==============================] - 2s 301us/step - loss: 0.4042 - acc: 0.8333
Epo

7200/7200 [==============================] - 2s 275us/step - loss: 0.3979 - acc: 0.8365
Epoch 78/100
7200/7200 [==============================] - 2s 283us/step - loss: 0.4035 - acc: 0.8326
Epoch 79/100
7200/7200 [==============================] - 2s 279us/step - loss: 0.4010 - acc: 0.8364
Epoch 79/100
7200/7200 [==============================] - 2s 267us/step - loss: 0.3979 - acc: 0.8360
Epoch 79/100
7200/7200 [==============================] - 2s 273us/step - loss: 0.4040 - acc: 0.8325
Epoch 80/100
7200/7200 [==============================] - 2s 282us/step - loss: 0.4008 - acc: 0.8340
Epoch 80/100
7200/7200 [==============================] - 2s 262us/step - loss: 0.3984 - acc: 0.8357
Epoch 80/100
7200/7200 [==============================] - 2s 283us/step - loss: 0.4013 - acc: 0.8353
Epoch 80/100
7200/7200 [==============================] - 2s 276us/step - loss: 0.4031 - acc: 0.8342
Epoch 81/100
7200/7200 [==============================] - 2s 284us/step - loss: 0.3999 - acc: 0.8332
Epo

7200/7200 [==============================] - 2s 325us/step - loss: 0.3977 - acc: 0.8358
Epoch 96/100
7200/7200 [==============================] - 2s 317us/step - loss: 0.3999 - acc: 0.8333
Epoch 97/100
7200/7200 [==============================] - 2s 331us/step - loss: 0.4008 - acc: 0.8356
Epoch 97/100
7200/7200 [==============================] - 2s 321us/step - loss: 0.4004 - acc: 0.8346
Epoch 98/100
7200/7200 [==============================] - 2s 333us/step - loss: 0.4009 - acc: 0.8362
Epoch 98/100
7200/7200 [==============================] - 2s 318us/step - loss: 0.3979 - acc: 0.8364
Epoch 98/100
7200/7200 [==============================] - 2s 314us/step - loss: 0.4034 - acc: 0.8308
Epoch 99/100
7200/7200 [==============================] - 2s 346us/step - loss: 0.4004 - acc: 0.8354
Epoch 99/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.3978 - acc: 0.8362
Epoch 99/100
7200/7200 [==============================] - 2s 337us/step - loss: 0.3999 - acc: 0.8336
Epo

7200/7200 [==============================] - 1s 176us/step - loss: 0.4069 - acc: 0.8343
Epoch 26/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.4004 - acc: 0.8360
Epoch 27/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.4070 - acc: 0.8339
Epoch 27/100
7200/7200 [==============================] - 1s 179us/step - loss: 0.4000 - acc: 0.8362
Epoch 28/100
7200/7200 [==============================] - 1s 176us/step - loss: 0.4064 - acc: 0.8340
Epoch 28/100
7200/7200 [==============================] - 1s 192us/step - loss: 0.3997 - acc: 0.8361
Epoch 29/100
7200/7200 [==============================] - 1s 190us/step - loss: 0.4066 - acc: 0.8337
Epoch 29/100
7200/7200 [==============================] - 1s 179us/step - loss: 0.3995 - acc: 0.8376
Epoch 30/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.4061 - acc: 0.8340
Epoch 30/100
7200/7200 [==============================] - 1s 177us/step - loss: 0.4002 - acc: 0.8357
Epo

7200/7200 [==============================] - 1s 183us/step - loss: 0.3940 - acc: 0.8378
Epoch 64/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.4047 - acc: 0.8315
Epoch 65/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3945 - acc: 0.8361
Epoch 65/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.4045 - acc: 0.8342
Epoch 66/100
7200/7200 [==============================] - 1s 188us/step - loss: 0.3942 - acc: 0.8375
Epoch 66/100
7200/7200 [==============================] - 2s 216us/step - loss: 0.4042 - acc: 0.8328
Epoch 67/100
7200/7200 [==============================] - 2s 215us/step - loss: 0.3940 - acc: 0.8378
Epoch 67/100
7200/7200 [==============================] - 2s 346us/step - loss: 0.4037 - acc: 0.8336
Epoch 68/100
7200/7200 [==============================] - 2s 344us/step - loss: 0.3931 - acc: 0.8376
Epoch 68/100
7200/7200 [==============================] - 3s 352us/step - loss: 0.3933 - acc: 0.8374
Epo

In [3]:
mean = accuracies.mean() #0.8422
variance = accuracies.std() #0.0183